In [1]:
import os
import numpy as np

import zipfile

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# data path initialization
BASE_DIR = '../'
TEXT_DATA_DIR = BASE_DIR + 'data/'
TEXT_DATA_FILE = "movie_reviews.csv"
HEADER = True

# parameters initialization
VALIDATION_SPLIT = 0.1
RANDOM_SEED = 42


def load_data():
    # function for loading data
    x = []
    y = []
    with open(os.path.join(TEXT_DATA_DIR, TEXT_DATA_FILE), "r", encoding="utf-8") as f:
        if HEADER:
            _ = next(f)
        for line in f:
            temp_y, temp_x = line.rstrip("\n").split(",", 1)
            x.append(temp_x.replace("'", ""))
            y.append(temp_y)
    return x, y

data, labels = load_data()
labels = np.asarray(labels, dtype='int8')

# spliting our original data on train and validation sets
data_train, data_val, labels_train, labels_val = \
    train_test_split(data, np.asarray(labels, dtype='int8'),
                     test_size=VALIDATION_SPLIT, random_state=RANDOM_SEED, stratify=labels)\
    

# initialize dictionary size and maximum sentence length
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40

# create a dictionary with Tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='#$%&()*+-/:;<=>@[\\]^{|}~\t\n,.!"?`')
tokenizer.fit_on_texts(data_train)

# replacing words with their indexes from our dictionary
X_train = tokenizer.texts_to_sequences(data_train)
X_val = tokenizer.texts_to_sequences(data_val)

# fit each sentence to max length
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)

Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: '../data/movie_reviews.csv'

In [ ]:
# path to embeddings file
EMBEDDINGS_DIR = BASE_DIR + 'embeddings'
EMBEDDINGS_FILE = 'glove.6B.50d.txt'

EMBEDDING_DIM = 50

# choose only 10000 words from our dictionary
first_10000 = {k: v for k, v in tokenizer.word_index.items() if v < 10000}

# upload embeddings
embeddings = {}
with zipfile.ZipFile(os.path.join(EMBEDDINGS_DIR, EMBEDDINGS_FILE+'.zip')) as myzip:
    with myzip.open(EMBEDDINGS_FILE) as f:
        for line in f:
            values = line.split()
            word = values[0].decode('UTF-8')
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs
        del values, word, coefs, line

In [ ]:
# prepare embeddings matrix where each row is word index

embedding_matrix = np.zeros((tokenizer.num_words, EMBEDDING_DIM))
for word, i in first_10000.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Model
from keras.layers import Input, Lambda
from keras.layers import MaxPooling1D, LSTM

NAME = "char_cnn_ohe"
# инициализация входа
in_sentence = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
# Lambda слой для ohe преобразования
embedded = Lambda(ohe, output_shape=lambda x: (x[0], x[1], vocab_size), arguments={"sz": vocab_size})(in_sentence)
block = embedded
# свертки с MaxPooling
for i in range(3):
    block = Conv1D(activation="relu", filters=100, kernel_size=4, padding="valid")(block)
    if i == 0:
        block = MaxPooling1D(pool_size=5)(block)
# LSTM ячейка
block = LSTM(128, dropout=0.1, recurrent_dropout=0.1)(block)
block = Dense(100, activation='relu')(block)
block = Dense(1, activation='sigmoid')(block)
# собираем модель
model = Model(inputs=in_sentence, outputs=block)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary()

#model.fit(X_train, labels_train, validation_data=[X_test, labels_test],
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])